In [1]:
import pandas as pd
import numpy as np
import torch

In [2]:
print(torch.cuda.device_count())

0


In [3]:
# read in the data
path = '/home/jimmy/CILProject22/data/raw/train_split_0.csv'
val_path = '/home/jimmy/CILProject22/data/raw/test_split_0.csv'

In [4]:
df = pd.read_csv(path)

In [5]:
df

,Id,Prediction
0,r44_c1,4
1,r61_c1,3
2,r72_c1,3
3,r86_c1,5
4,r108_c1,3
...,...,...
941556,r9970_c1000,5
941557,r9976_c1000,5
941558,r9990_c1000,4
941559,r9992_c1000,5


In [6]:
from src.data.Contrastive.graph_datamodule import Triplet_Dataset
dataset = Triplet_Dataset(file_path=path, n_users=1000, n_items=10000, k=2)


Computing Neighborhood
0 neighborhood
1 neighborhood


/home/jimmy/CILProject22/src/data/Contrastive/graph_datamodule.py:55: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at  ../aten/src/ATen/SparseCsrTensorImpl.cpp:68.)
  self.k_neighborhood = self.k_neighborhood.to_sparse_csr()


In [7]:
dataset.__getitem__(0)

Retrieving neighbors 0.0002484321594238281
Sampling neighbors0.0006475448608398438
Sampling negative edges 1.2159347534179688e-05
Sampling negative neighbors 0.0017616748809814453


(tensor(0),
 tensor(1043),
 tensor([6636]),
 tensor([365]),
 tensor([634, 260, 380, 681, 335]),
 tensor([9861, 3662, 9327, 2259, 4627]),
 tensor([2606, 3379, 9672,  869, 8359]),
 tensor([6835,  429, 1885, 4295, 9056]))

In [11]:
from torch.utils.data import DataLoader
#from src.data.Contrastive.graph_datamodule import collate_tuples
dataloader = DataLoader(dataset, batch_size=16, shuffle=True, num_workers=4)

In [12]:
import time
t0 = time.time()
for i in dataloader:
    t1 = time.time()
    break
t1-t0

Retrieving neighbors 0.00011038780212402344Retrieving neighbors 0.00018668174743652344Retrieving neighbors 0.0001430511474609375Retrieving neighbors 0.0001430511474609375



Sampling neighbors0.003186464309692383Sampling neighbors0.002755880355834961Sampling neighbors0.0029082298278808594
Sampling neighbors0.003597736358642578


Sampling negative edges 0.0013048648834228516Sampling negative edges 0.0013735294342041016

Sampling negative edges 0.0015916824340820312
Sampling negative edges 0.0013625621795654297Sampling negative neighbors 0.0035402774810791016Sampling negative neighbors 0.003229856491088867


Sampling negative neighbors 0.003648519515991211Retrieving neighbors 5.030632019042969e-05Retrieving neighbors 3.075599670410156e-05


Sampling negative neighbors 0.005117893218994141Retrieving neighbors 3.62396240234375e-05Sampling neighbors0.0012645721435546875
Sampling neighbors0.0012331008911132812


Retrieving neighbors 3.9577484130859375e-05Sampling neighbors0.00139212608337402

0.42121338844299316

In [3]:
val_dataset = Triplet_Dataset(file_path=val_path, n_users=1000, n_items=10000,k=2)
val_dataloader = DataLoader(val_dataset, batch_size=4, num_workers=4)

In [34]:
# Create Model Object
x, y, x_neighbors, y_neighbors, x_negative, y_negative, negative_x, negative_y = dataset.__getitem__(0)

In [1]:
from src.models.CSE.model import model
import os
import time
model = model()

In [3]:
EPOCH = 50
from tqdm import tqdm
import torch
optimizer = model.configure_optimizers()
run_id = time.strftime("%Y%m%d-%H%M%S")
log_dir = f"/home/jimmy/CILProject22/reports/logs/{run_id}_pretrain"
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
    # Create logging file

In [ ]:
for i in range(EPOCH):
    print(f'Epoch {i}')
    train_loss = 0
    model.train()
    for batch in tqdm(dataloader):
        loss = model.training_step(batch, batch_idx=0)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        train_loss += loss
    print(f'Train Loss: {train_loss/len(dataloader)}')
    model.eval()
    print(f'Validating')
    model.eval()
    with torch.no_grad():
        val_loss = 0
        for batch in tqdm(val_dataloader):
            loss = model.val_step(batch, batch_idx=0)
            val_loss += loss
        print(f'Val Loss: {val_loss/len(val_dataloader)}')

    torch.save({
        'epoch': i,
        'model_state_dict': model.state_dict(),
        'loss': train_loss
    }, log_dir+f'/model_{i}.pth')

In [8]:
phi, phi_IC, phi_UC = model(x)

In [9]:
phi.size()

torch.Size([10])

In [10]:
phin, phin_IC, phin_UC = model(x_neighbors)

In [11]:
phin.size()

torch.Size([1, 10])

In [15]:
x_neighbors.size()
import torch

In [24]:
torch.inner(phi, phin)

tensor([2.6093], grad_fn=<ReshapeAliasBackward0>)

In [26]:
phi_n, phi_n_IC, phi_n_UC = model(x_negative)

In [28]:
torch.inner(phi_n, phi)

tensor([ 0.0206, -3.3579, -2.3495, -1.2092, -1.9400],
       grad_fn=<ReshapeAliasBackward0>)

In [29]:
torch.inner(phi_n[0], phi)

tensor(0.0206, grad_fn=<ReshapeAliasBackward0>)

In [37]:
y > 1000

tensor(True)

In [31]:
x, y = y, x

In [32]:
x

5

In [53]:
import torch.nn.functional as F

In [57]:
test_x, test_IC = torch.ones((32, 5, 5)), torch.ones((32, 5, 5))

In [60]:
F.logsigmoid(torch.sum(torch.mul(test_x, test_IC), dim=2)).size()

torch.Size([32, 5])

In [ ]:
test_x, test_IC = torch.ones((32, 5)), torch.ones((32, 5))